In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re
import time
import numpy as np

In [2]:
ano = '202223'
url = 'https://fbref.com/en/comps/9/2022-2023/schedule/2022-2023-Premier-League-Scores-and-Fixtures'
playoff = True

In [3]:
df_update = pd.read_html(url, extract_links="all")
df_update = df_update[0]

df_update.columns = [col[0] for col in df_update.columns]

df_update_getSecondTuple = ['Match Report']

for column in df_update.columns:
    if column not in df_update_getSecondTuple:
        df_update[column] = df_update[column].apply(lambda x: x[0])
    else:
        df_update[column] = df_update[column].apply(lambda x: x[1])
        
#if playoff:
#    df_update = df_update[:-2]
#    df_update.drop(['Rodada'],inplace=True,axis=1)
            
df_update['goals_home'] = df_update['Score'].str.split('–').str[0]
df_update['goals_away'] = df_update['Score'].str.split('–').str[1]
df_update.drop(['Score','Notes'],axis=1,inplace=True)
values = ['None']
df_update = df_update[df_update['Match Report'].isna() == False].reset_index(drop=True) 
df_update.rename({'xG': 'xG_home', '(': 'xG_away', 'Home': 'home', 'Away': 'away', 'Attendance': 'Assistance', 'Wk': 'Matchday'}, axis=1, inplace=True)

In [4]:
df_update.to_csv(f'fbref/dataset_{ano}.csv')


In [ ]:
pd.read_html(str(table))[0]

In [6]:
try:
    df = pd.read_csv(f'fbref/dataset_{ano}.csv')
    #df.drop('Unnamed: 0',inplace=True)
except:
    df = df_update.copy()
    
#df_update['Assistance'] =  df_update['Assistance'].str.replace('.', '').replace('', 0)
#df_update['Assistance'] = df_update['Assistance'].astype(int)
#df['Assistance'] = df_update['Assistance']

for index, row in df[df['Match Report'].str.startswith('/en/stathead/')].iterrows():
    df.loc[index] = df_update.loc[index]

for index, row in df.iterrows():
    
    if row['Match Report'].startswith('/en/matches/') and ('player1_home' not in df.columns or pd.isna(df.loc[index,'player1_home'])):
    
        start = time.time()

        url = row['Match Report']
        print(url)
        r = requests.get('http://fbref.com' + url)
        soup = BeautifulSoup(r.content, 'html.parser')
        tables = soup.find_all('table')
        try:
            table = tables[0]
            table2 = tables[1] 
        except Exception:
            indice = index
        try:
            home_players = pd.read_html(str(table))[0].drop(11)
            away_players = pd.read_html(str(table2))[0].drop(11)
        except:
            continue
        df.loc[index,'formacao_home'] = home_players.columns[0].split('(')[1][:-1]
        df.loc[index,'formacao_away'] = away_players.columns[0].split('(')[1][:-1]
        home_players = home_players.transpose().reset_index().drop('index',axis=1)
        away_players = away_players.transpose().reset_index().drop('index',axis=1)

        print(f'meio: {time.time() - start}')
        
        stats_div = soup.find('div', {'id': 'team_stats_extra'})

        # extrai o conteúdo da div
        stats_text = stats_div.text

        linhas = stats_text.split('\n')
        palavras_chave = ['Fouls', 'Corners', 'Crosses', 'Touches', 'Interceptions', 'Tackles', 'Aerials Won', 'Clearances', 'Offsides', 'Goal Kicks', 'Throw Ins', 'Long Balls']

        lista_strings = [string for string in linhas for palavra in palavras_chave if palavra in string]

        nova_lista = []
        for string in lista_strings:
            match = re.match(r'^(\d+)([a-zA-Z\s]+)(\d+)$', string)
            if match:
                nova_lista.append([int(match.group(1)), match.group(2).strip(), int(match.group(3))])
        
        for x in nova_lista:
            df.loc[index, f"{x[1]}_home"] = x[0]
            df.loc[index, f"{x[1]}_away"] = x[2]
            
            
        cards = {
            'home_team': {
                'yellow_card': 0,
                'red_card': 0,
                'yellow_red_card': 0
            },
            'away_team': {
                'yellow_card': 0,
                'red_card': 0,
                'yellow_red_card': 0
            }
        }

        home_team_cards_div = soup.find_all('div', class_='cards')[0]
        home_team_cards_spans = home_team_cards_div.find_all('span')
        for span in home_team_cards_spans:
            cards['home_team'][span['class'][0]] += 1

        away_team_cards_div = soup.find_all('div', class_='cards')[1]
        away_team_cards_spans = away_team_cards_div.find_all('span')
        for span in away_team_cards_spans:
            cards['away_team'][span['class'][0]] += 1
        
        df.loc[index, 'yellow_card_home'] = cards['home_team']['yellow_card']
        df.loc[index, 'red_card_home'] = cards['home_team']['red_card']
        df.loc[index, 'yellow_red_card_home'] = cards['home_team']['yellow_red_card']
        df.loc[index, 'yellow_card_away'] = cards['away_team']['yellow_card']
        df.loc[index, 'red_card_away'] = cards['away_team']['red_card']
        df.loc[index, 'yellow_red_card_away'] = cards['away_team']['yellow_red_card']
        
        for i in range(len(home_players.columns) - 1):
            column_name = f'jogador{i+1}_home'
            df.loc[index, column_name] = home_players.iloc[1, i]
        for i in range(len(away_players.columns) - 1):
            column_name = f'jogador{i+1}_away'
            df.loc[index, column_name] = away_players.iloc[1, i]

        print(f'fim: {time.time() - start}')
        if time.time() - start < 6.0:
            time.sleep(6.1 - (time.time() - start))
            
df.to_csv(f'dataset_{ano}.csv')
df.head()

/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League


C:\Users\user\AppData\Local\Temp\ipykernel_24300\2944083324.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  home_players = pd.read_html(str(table))[0].drop(11)
C:\Users\user\AppData\Local\Temp\ipykernel_24300\2944083324.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  away_players = pd.read_html(str(table2))[0].drop(11)


meio: 1.0216915607452393
fim: 1.0745728015899658
/en/matches/6713c1dc/Fulham-Liverpool-August-6-2022-Premier-League


C:\Users\user\AppData\Local\Temp\ipykernel_24300\2944083324.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  home_players = pd.read_html(str(table))[0].drop(11)
C:\Users\user\AppData\Local\Temp\ipykernel_24300\2944083324.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  away_players = pd.read_html(str(table2))[0].drop(11)


meio: 1.063880205154419
fim: 1.107003927230835
/en/matches/09d8a999/Tottenham-Hotspur-Southampton-August-6-2022-Premier-League


...

C:\Users\user\AppData\Local\Temp\ipykernel_24300\2944083324.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  home_players = pd.read_html(str(table))[0].drop(11)
C:\Users\user\AppData\Local\Temp\ipykernel_24300\2944083324.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  away_players = pd.read_html(str(table2))[0].drop(11)


meio: 1.9675674438476562
fim: 2.0110440254211426
/en/matches/e7e969e9/Brentford-Manchester-City-May-28-2023-Premier-League


C:\Users\user\AppData\Local\Temp\ipykernel_24300\2944083324.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  home_players = pd.read_html(str(table))[0].drop(11)
C:\Users\user\AppData\Local\Temp\ipykernel_24300\2944083324.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  away_players = pd.read_html(str(table2))[0].drop(11)


meio: 1.2827653884887695
fim: 1.32016921043396


,Unnamed: 0,Matchday,Day,Date,Time,home,xG_home,xG_away,away,Assistance,Venue,Referee,Match Report,goals_home,goals_away,formacao_home,formacao_away,Fouls_home,Fouls_away,Corners_home,Corners_away,Crosses_home,Crosses_away,Touches_home,Touches_away,Tackles_home,Tackles_away,Interceptions_home,Interceptions_away,Aerials Won_home,Aerials Won_away,Clearances_home,Clearances_away,Offsides_home,Offsides_away,Goal Kicks_home,Goal Kicks_away,Throw Ins_home,Throw Ins_away,Long Balls_home,Long Balls_away,yellow_card_home,red_card_home,yellow_red_card_home,yellow_card_away,red_card_away,yellow_red_card_away,jogador1_home,jogador2_home,jogador3_home,jogador4_home,jogador5_home,jogador6_home,jogador7_home,jogador8_home,jogador9_home,jogador10_home,jogador11_home,jogador12_home,jogador13_home,jogador14_home,jogador15_home,jogador16_home,jogador17_home,jogador18_home,jogador19_home,jogador1_away,jogador2_away,jogador3_away,jogador4_away,jogador5_away,jogador6_away,jogador7_away,jogador8_away,jogador9_away,jogador10_away,jogador11_away,jogador12_away,jogador13_away,jogador14_away,jogador15_away,jogador16_away,jogador17_away,jogador18_away,jogador19_away
0,0,1,Fri,2022-08-05,20:00,Crystal Palace,1.2,1.0,Arsenal,"25,286",Selhurst Park,Anthony Taylor,/en/matches/e62f6e78/Crystal-Palace-Arsenal-Au...,0,2,4-2-3-1,4-3-3,16.0,11.0,3.0,5.0,19.0,11.0,726.0,599.0,18.0,29.0,8.0,9.0,10.0,14.0,18.0,24.0,1.0,2.0,4.0,2.0,25.0,14.0,83.0,59.0,1.0,0.0,0.0,2.0,0.0,0.0,Vicente Guaita,Tyrick Mitchell,Marc Guéhi,Jordan Ayew,Eberechi Eze,Wilfried Zaha,Jeffrey Schlupp,Joachim Andersen,Nathaniel Clyne,Odsonne Édouard,Cheick Doucouré,Sam Johnstone,Joel Ward,Luka Milivojević,Jean-Philippe Mateta,Will Hughes,Malcolm Ebiowei,Chris Richards,Jaïro Riedewald,Aaron Ramsdale,Ben White,Thomas Partey,Gabriel Dos Santos,Bukayo Saka,Martin Ødegaard,Gabriel Jesus,Gabriel Martinelli,William Saliba,Granit Xhaka,Oleksandr Zinchenko,Matt Turner,Kieran Tierney,Eddie Nketiah,Rob Holding,Cédric Soares,Nicolas Pépé,Albert Sambi Lokonga,Reiss Nelson
1,1,1,Sat,2022-08-06,12:30,Fulham,1.2,1.2,Liverpool,"22,207",Craven Cottage,Andy Madley,/en/matches/6713c1dc/Fulham-Liverpool-August-6...,2,2,4-2-3-1,4-3-3,7.0,9.0,4.0,4.0,19.0,18.0,474.0,784.0,24.0,11.0,10.0,10.0,23.0,13.0,31.0,16.0,4.0,4.0,8.0,5.0,22.0,35.0,77.0,94.0,2.0,0.0,0.0,0.0,0.0,0.0,Marek Rodák,Kenny Tete,Tosin Adarabioyo,Harrison Reed,Neeskens Kebano,Aleksandar Mitrović,Tim Ream,Bobby Reid,Andreas Pereira,João Palhinha,Antonee Robinson,Bernd Leno,Shane Duffy,Tom Cairney,Manor Solomon,Rodrigo Muniz,Kevin Mbabu,Tyrese Francois,Luke Harris,Alisson,Fabinho,Virgil van Dijk,Thiago Alcántara,Roberto Firmino,Mohamed Salah,Jordan Henderson,Luis Díaz,Andrew Robertson,Joël Matip,Trent Alexander-Arnold,Adrián,Joe Gomez,James Milner,Harvey Elliott,Darwin Núñez,Fabio Carvalho,Stefan Bajcetic,Sepp van den Berg
2,2,1,Sat,2022-08-06,15:00,Tottenham,1.5,0.5,Southampton,"61,732",Tottenham Hotspur Stadium,Andre Marriner,/en/matches/09d8a999/Tottenham-Hotspur-Southam...,4,1,3-4-3,5-3-2,11.0,6.0,10.0,2.0,24.0,18.0,709.0,554.0,24.0,14.0,3.0,13.0,13.0,11.0,20.0,21.0,2.0,0.0,6.0,4.0,20.0,14.0,85.0,61.0,3.0,0.0,0.0,0.0,0.0,0.0,Hugo Lloris,Pierre Højbjerg,Son Heung-min,Harry Kane,Emerson,Eric Dier,Cristian Romero,Ryan Sessegnon,Dejan Kulusevski,Rodrigo Bentancur,Ben Davies,Fraser Forster,Matt Doherty,Davinson Sánchez,Bryan Gil,Ivan Perišić,Djed Spence,Lucas Moura,Clément Lenglet,Gavin Bazunu,Kyle Walker-Peters,Oriol Romeu,Joe Aribo,James Ward-Prowse,Adam Armstrong,Moussa Djenepo,Mohammed Salisu,Jan Bednarek,Yan Valery,Roméo Lavia,Alex McCarthy,Jack Stephens,Che Adams,Nathan Redmond,Romain Perraud,Stuart Armstrong,Mohamed Elyounoussi,Ibrahima Diallo
3,3,1,Sat,2022-08-06,15:00,Newcastle Utd,1.7,0.3,Nott'ham Forest,"52,245",St James' Park,Simon Hooper,/en/matches/1ac96eb4/Newcastle-United-Nottingh...,2,0,4-3-3,3-4-1-2,9.0,14.0,11.0,1.0,25.0,9.0,634.0,475.0,19.0,15.0,11.0,10.0,12.0,16.0,14.0,37.0,2.0,0.0,2.0,12.0,21.0,26.0,64.0,70.0,0.0,0.0,0.0,3.0,0.0,0.0,Nick P

In [ ]:
df.to_csv(f'dataset_{ano}.csv')
df

In [ ]:
### 538
df_538 = pd.read_csv('https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv')
df_538 = df_538[df_538['league'] == 'Portuguese Liga']
df_538 = df_538[df_538['season'] == 2022] 


In [ ]:
df_538['league'].unique()

In [ ]:
df_update = pd.read_html('https://fbref.com/pt/comps/32/cronograma/Primeira-Liga-Resultados-e-Calendarios#sched_2022-2023_32_1', extract_links="all")
df_update = df_update[0]

In [ ]:
import numpy as np
df = pd.read_csv('output_dataset_exemplo.csv')
df.set_index('Unnamed: 0', inplace=True)
df_update['Assistance'] =  df_update['Assistance'].str.replace('.', '').replace('', np.nan)
df_update['Assistance'] = df_update['Assistance'].astype(float)
df['Assistance'] = df_update['Assistance']

In [ ]:
df_update.columns = [col[0] for col in df_update.columns]

df_update_getSecondTuple = ['Relatório da Partida']

for column in df_update.columns:
    if column not in df_update_getSecondTuple:
        df_update[column] = df_update[column].apply(lambda x: x[0])
    else:
        df_update[column] = df_update[column].apply(lambda x: x[1])

In [ ]:
df2 = df_update[df_update['Assistance'].isna() == False]#.reset_index() 
#

In [ ]:
df

In [ ]:
df_diff = df.compare(df_update)

# Seleciona as linhas que são diferentes em df e df_update
row_idx = df_diff.index[df_diff.ne("").any(axis=1)]

# Substitui as linhas em df pelos valores atualizados em df_update
#df.loc[row_idx] = df_update.loc[row_idx]

In [ ]:
df1 = pd.read_csv(f'fbref/dataset_201819.csv')
df2 = pd.read_csv(f'fbref/dataset_201920.csv')
df3 = pd.read_csv(f'fbref/dataset_202021.csv')
df4 = pd.read_csv(f'fbref/dataset_202122.csv')
df5 = pd.read_csv(f'fbref/dataset_202223.csv')
df = pd.concat([df1,df2,df3,df4,df5])
df['home']